In [15]:
import requests
import json
import pickle
import pandas as pd
from time import sleep
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from collections import Counter
from sklearn.metrics.pairwise import cosine_distances
from collections import defaultdict
import numpy as np


In [2]:
df_vid = pd.read_csv('ods_videos.csv')

In [244]:
df_vid['publishedAt'] = pd.to_datetime(a).apply(lambda x: str(x).split(' ')[0])

In [245]:
df_vid

,youtube_id,channel,title,publishedAt,description,url
0,46nnbxEZNw8,Компьютерные науки,068. Как извлечь из сырых данных пользу для му...,2018-07-03,В рамках конференции «Data&Science: маркетинг»...,https://www.youtube.com/watch?v=46nnbxEZNw8
1,RnUcRYAbWtY,Компьютерные науки,001. Прикладные задачи оптимизации – М.В. Бацын,2015-04-10,NaN,https://www.youtube.com/watch?v=RnUcRYAbWtY
2,frLW8UVp_Ik,Компьютерные науки,005. Тематическое моделирование на пути к разв...,2016-09-27,NaN,https://www.youtube.com/watch?v=frLW8UVp_Ik
3,7lerRDCZrhU,Компьютерные науки,Запись тренировки ML 29.07.2017,2017-07-29,Описание:\nРим Шаяхметов — Предсказание сердеч...,https://www.youtube.com/watch?v=7lerRDCZrhU
4,Zco55S3RNU0,Компьютерные науки,058. Круглый стол «Как анализ данных меняет м...,2018-02-21,В рамках конференции Яндекса «Большие данные в...,https://www.youtube.com/watch?v=Zco55S3RNU0
5,3JpkBRJmo6Y,Компьютерные науки,018. Астрономия в эпоху больших обзоров неба –...,2016-12-26,За последние 15 лет благодаря появлению больши...,https://www.youtube.com/watch?v=3JpkBRJmo6Y
6,uWWQClHVwEc,Компьютерные науки,003 Запись трансляции научно-технического семи...,2016-05-24,24 мая в московском офисе Яндекса выступит оди...,https://www.youtube.com/watch?v=uWWQClHVwEc
7,fBVMLqi29XQ,Компьютерные науки,009. Коллективный поиск: подходы к изучению со...,2014-11-21,Традиционная модель интернет поиска себя исчер...,https://www.youtube.com/watch?v=fBVMLqi29XQ
8,JJCO7eWCy-M,Компьютерные науки,DataFest 5 - Lumiere hall 2 - Запись трансляции,2018-04-28,Программа:\n\n00:02:30 Pitch-сессия\nУчастники...,https://www.youtube.com/watch?v=JJCO7eWCy-M
9,egjoKTAYi7g,Компьютерные науки,002. Что такое ШАД – Станислав Федотов,2017-04-20,Станислав расскажет о правилах поступления и о...,https://www.youtube.com/watch?v=egjoKTAYi7g


In [247]:
df_vid.to_csv('ods_videos.csv', index=None)

In [11]:
id2title = {idx:title for idx, title in df_vid[['youtube_id', 'title']].values}


In [5]:
id2trans = pickle.load(open('id2transcripts.pkl', 'rb'))

In [31]:
id2trans.keys()

dict_keys(['2c7HGBuXOZ8', 'pwO8aegWSbw', 'Mw2vdYv4ups', 'EYqDuF_dPzU', 'OwXfOqaGhS0', 'sARwqGY5l44', 'dCIM1EZ1WX4', 'PZvIql_DIcs', 'TD2PF6TZcx0', 'ZdcvQYygtDA', 'xkN3WyNeuU0', 'VkIe4znmTow', 'NFeak71ewD8', '6ZlfYm2xgtY', 'l2jN471TEQg', 'vqtdZx0mS3A', 'pruqXRLy_UM', 'yTMcZ1Ly8Jk', '4RxAbBqqsqE', 'EIvdPxBYSGY', 'u6FBlIsClx8', 'C_XUl52S3KU', 'bJ8Ro1OVaSM', 'sz_Gvtx2jBA', 'zjsG81t2VkU', 'b7-5vH6G3XI', 'JDSK39dr2rc', '5oqdfBqok94', '07_ghIkP7Ig', '5kwEL-a8PdQ', 'mDFf7W8O_kU', 'm4yxsBMBgtM', 'WCE9hhPbCmc', '7sZeTxIrnxs', 'BCYdTvg-lks', 'u_jtW028pAA', 'vtG2GmORbEA', '86rSzFAu0L8', 'xglXoAIxzkc', '2dhwd-dYZhI', 'FahVtbxsEzA', 'IPG4-p7VVkI', 'omwcHWVa3WI', '1CwM7X3vEpw', 'fghcOhTo3JE', 'nPcSGlXiiMM', 'HsxkQzOu7oE', 'ZdHqS0u1G_c', 'IaFnLlWVOm0', '4j82U9iYrlk', 'J05kQdgn6k0', 'wbigj-HsTt4', 'kzkJNpSSaNE', 'TMOrMhinpzQ', 'QL4_AWN7V6E', 'LKqo-KzqUVU', 'JHzY7LWtEs8', '3Ru4OBPIcIw', 'k4mvy8v1Wns', '73P0T3_nGd4', 'qgFRmlKZaSc', '461_3NJe9gY', 'F-j0G0lrjFw', 'kRjC7nzSHWk', 'a_Bh0lLq36E', 'WaF8oJCbv6Q',

In [10]:
id2trans['2c7HGBuXOZ8'][:30]

[{'duration': 3.38, 'start': 6.87, 'text': 'такси недавно запустили серию'},
 {'duration': 5.15, 'start': 9.35, 'text': 'мероприятий д.т.н.'},
 {'duration': 5.9, 'start': 10.25, 'text': 'сайнс который людей от науки ра'},
 {'duration': 4.2,
  'start': 14.5,
  'text': 'про свои задачи которых анализ данных'},
 {'duration': 4.44,
  'start': 16.15,
  'text': 'мог бы помочь а людей развивается да и'},
 {'duration': 4.22, 'start': 18.7, 'text': 'касается могли бы узнать новую'},
 {'duration': 7.32,
  'start': 20.59,
  'text': 'предметную область для себя и в ней вот'},
 {'duration': 8.56,
  'start': 22.92,
  'text': 'покопаться и первый метод был про физику'},
 {'duration': 5.219,
  'start': 27.91,
  'text': 'высоко физика высоких энергий и в'},
 {'duration': 8.42,
  'start': 31.48,
  'text': 'частности на этом do it when silence of'},
 {'duration': 11.731,
  'start': 33.129,
  'text': 'митапе было выдано задание про которая'},
 {'duration': 10.96,
  'start': 39.9,
  'text': 'богдан который

In [191]:
# склеиваем кусочки транскриптов в куски длинной 30 сек
pieces = []

for key in id2trans:
    text = []
    duration = 0
    start = None
    for piece in id2trans[key]:
        if not piece['text']:
            continue
        text.append(piece['text'])
        duration += piece['duration']
        if not start:
            start = piece['start']
        
        if duration > 30:
            pieces.append((key, ' '.join(text), start, ))
            text = []
            duration = 0
            start = None



In [192]:
index2id = {i:idx for i, (idx, _, _) in enumerate(pieces)}


In [193]:
texts = [p[1] for p in pieces]

In [118]:
texts_norm = [normalize(p[1]) for p in pieces]

In [194]:
texts_norm = [normalize_mystem(p[1]) for p in pieces]

In [199]:
texts_norm = [' '.join(text) for text in texts_norm]

In [200]:
pickle.dump(texts_norm, open('transcripts.pkl', 'wb'))

In [228]:
descriptions[0]

'в рамка конференция Data Science маркетинг илья чухляев OWOX и екатериа курочкин м видео рассказывать как объединять несемплированный данные делать маркетинг м видео эффективно что такой Google BigQuery и как он помогать в объединение онлайн данные и офлайн данные Google Data Prep как обрабатывать и подготовить большой объем данный к анализ кейс м видео как повышать эффективность маркетинг анализировать омниканальный взаимодействие с клиент скачать презентация https yadi sk i Nb24Q3Dc3YivZn илья чухляев региональный директор OWOX по регион CIS заниматься онлайн аналитика больше шесть год защищать диссертация по веб аналитика в сша учиться у авинаш кошик ранее возглавлять отдел аналитик Adventum где работать с крупный клиент из разный отрасль илья сертифицированный консультант Google Analytics и партнер яндекс по обучение преподавать онлайн аналитика на специализированный курсы в tceh com екатерина курочкин руководитель направление интернет маркетинг м видео заниматься интернет маркети

In [196]:
index2id = {i:idx for i, (idx, _, _) in enumerate(pieces)}

In [230]:
titles = df_vid['title'].apply(str).apply(normalize_mystem).values
descriptions = df_vid['description'].apply(str).apply(normalize_mystem).values

In [222]:
id2index_table = {idx:i for i, idx in enumerate(df_vid['youtube_id'].values)}

In [223]:
id2title = {idx:title for idx, title in df_vid[['youtube_id', 'title']].values}

In [224]:
times = [round(p[2]) for p in pieces]

pickle.dump(times, open('times.pkl', 'wb'))

In [231]:
pickle.dump(id2index_table, open('id2index_table.pkl', 'wb'))
pickle.dump(id2title, open('id2title.pkl', 'wb'))
pickle.dump(index2id, open('index2id.pkl', 'wb'))
pickle.dump(titles, open('titles.pkl', 'wb'))
pickle.dump(descriptions, open('descriptions.pkl', 'wb'))

In [182]:
id2index_table

{'-2UH0soWbZw': 340,
 '-8gnGgH9YLY': 345,
 '-FkyT4I43aI': 128,
 '-H11HNEOgvY': 260,
 '-HwjbfQ80WY': 862,
 '-J1V8fEzb2Q': 303,
 '-Kvn2ppfYRE': 49,
 '-NVQDWpMLKE': 390,
 '-OXeQFIugnI': 421,
 '-PnKega5D6k': 283,
 '-WWAcltTlRw': 27,
 '-YIAdMxrmnc': 351,
 '-e0oazB3kyw': 212,
 '-foNYaNQt6k': 435,
 '-jtqT5DihPc': 908,
 '-kQWXQ-N048': 853,
 '-psvOsqPx1g': 235,
 '-q6M59qS0FU': 63,
 '-tYO8EmeGsw': 615,
 '00qF2yMuRkQ': 162,
 '06CHSz_DQFQ': 170,
 '07_ghIkP7Ig': 284,
 '08VkuplUiKs': 478,
 '0I-GydFxXec': 566,
 '0K8XzOqBMus': 619,
 '0VdcTdHb3ZQ': 304,
 '0Yefpq-E_KQ': 788,
 '0_brTY6QgsA': 33,
 '0cBRVTGHdVQ': 826,
 '0fdSDdYeJfE': 272,
 '0hPY0gSOFro': 627,
 '0qHXNeuNOAE': 850,
 '0sg15V669qM': 605,
 '0wH2IiUa4hQ': 461,
 '0wgAZL9yV0I': 159,
 '17aRczhTpC4': 139,
 '1Axti81lRmw': 671,
 '1Chk1Mi-yZ0': 451,
 '1CwM7X3vEpw': 425,
 '1HrkBzLBJQg': 323,
 '1LcdA0Y7IEk': 506,
 '1Mxcylr3dd4': 844,
 '1ORhXyCCs4s': 609,
 '1OUv73VaZes': 233,
 '1aXJpL1l8wQ': 160,
 '1atJMUxaH9o': 387,
 '1kROiEt3f1U': 333,
 '1pvXww9tSCA': 4

In [229]:
from string import punctuation, digits
from pymorphy2 import MorphAnalyzer
from pymystem3 import Mystem
from nltk.corpus import stopwords

stops = stopwords.words('russian')
m = Mystem()

punct = punctuation+'«»—…“”*№–'
def normalize(text):
    words = [word.strip(punct) for word in text.lower().split()]
    words = [m.parse(word)[0].normal_form for word in words if word and word not in stops]

    return words

def normalize_mystem(text):
    words = m.lemmatize(text.lower())
    words = [word for word in words if word and word.isalnum()]
    
    return ' '.join(words)

Строим тфидф по транскриптам

In [144]:
tfidf = TfidfVectorizer(min_df=5,max_features=25000, )
svd = TruncatedSVD(200)

index = svd.fit_transform(tfidf.fit_transform(texts_norm))

Транформим какой-то текст в вектор и считаем косинус до остальных транскриптов. Аггрегируем близости по ид_видео 

In [155]:
accum = defaultdict(list)
# qu
sims = cosine_distances(svd.transform(tfidf.transform([' '.join(normalize_mystem('как управлять командой датасаентистов'))])), index)[0]

for arg, sim in enumerate(sims):
    accum[index2id[arg]].append((arg, sim))

Находим наиболее близкий видос по среднему косинуса между нашим текстом и транскриптами этого видоса
Находим для видоса наиболее близкий траскрипт

In [156]:
id2mean = [(key, np.min([x[1] for x in elements])) for key, elements in accum.items()]
id2maxind = {key:min(elements, key=lambda x: x[1])[0] for key, elements in accum.items()}

In [157]:
id2mean_s = sorted(id2mean, key=lambda x: x[1])

Показываем самые близкие

In [158]:
[(id2title[idx], texts_norm[id2maxind[idx]]) for idx, _ in id2mean_s[:5]]

[('Kaggle Machine Learning Mania 2016 Баскетбол —\xa0Смердов Антон',
  'непонятно что здесь поставлять целевой переменная и он мочь не понимать как ты вообще интерпретировать как это можно сделать вот например можно подавать строка как айди команда победитель оба команда проигрывать ну они хорошо бен нарисовать и параметр связывать с погибший команда и проигрывать команда целевой переменная мы нужно определять'),
 ('007. Андрей Кирьясов – Home Depot Search Relevance',
  'предложение объединяться в команда поскольку как бы другой есть я не быть я решать что достаточно интересно глупо работать в команда соглашаться достаточно быстро и весь дальнейший действие мы осуществлять уже состав команда команда состоять из три человек алексей шавкун алексей матвеев и я мы'),
 ('Kaggle Home Depot Search Relevance — Андрей Кирясов',
  'предложение объединяться в команда поскольку как бы другой есть я не быть я решать что достаточно интересно глупо работать в команда соглашаться достаточно быстро и

In [208]:
import gensim
ph = gensim.models.Phrases([t.split() for t in texts_norm])
p = gensim.models.phrases.Phraser(ph)

In [212]:


# texts_norm = [normalize(p[1]) for p in pieces]
ft = gensim.models.FastText([t.split() for t in texts_norm],sg=1, size=200)

In [213]:
ft.most_similar('натекин')

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.5/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('чекин', 0.8433210849761963),
 ('зигзаг', 0.8200596570968628),
 ('натан', 0.8196519017219543),
 ('пекин', 0.8152347803115845),
 ('нарезка', 0.8137025833129883),
 ('пьеса', 0.8127280473709106),
 ('чикин', 0.8004963397979736),
 ('072', 0.8004614114761353),
 ('нтв', 0.7958810329437256),
 ('горшочек', 0.7952461242675781)]

In [214]:

vocab = ft.wv.index2word
matrix = np.zeros((len(vocab), 200))

for i, word in enumerate(vocab):
    if word in vocab:
        matrix[i] = ft[word]

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


In [216]:
from sklearn.cluster import MiniBatchKMeans
import hdb

# cluster = MiniBatchKMeans(1000, n_init=2, verbose=1, max_no_improvement=300, reassignment_ratio=0.4)
cluster.fit(matrix)
cls = defaultdict(list)

for i, cl in enumerate(cluster.labels_):
    cls[cl].append(vocab[i])

f = open('cluster_ft.txt', 'w')
for cl in cls:
    f.write('### '+ str(cl) + ' ###\n')
    f.write('\n'.join(cls[cl]))
    f.write('\n\n')
f.close()

Init 1/2 with method: k-means++


/usr/local/lib/python3.5/dist-packages/sklearn/cluster/k_means_.py:1556: RuntimeWarning: init_size=300 should be larger than k=1000. Setting it to 3*k
  init_size=init_size)


Inertia for init 1/2: 200.239167
Init 2/2 with method: k-means++


/usr/local/lib/python3.5/dist-packages/sklearn/cluster/k_means_.py:1556: RuntimeWarning: init_size=300 should be larger than k=1000. Setting it to 3*k
  init_size=init_size)


Inertia for init 2/2: 185.454906
Minibatch iteration 1/19200: mean batch inertia: 1.986429, ewa inertia: 1.986429 
Minibatch iteration 2/19200: mean batch inertia: 1.990906, ewa inertia: 1.986476 
Minibatch iteration 3/19200: mean batch inertia: 2.417817, ewa inertia: 1.990989 
Minibatch iteration 4/19200: mean batch inertia: 1.920245, ewa inertia: 1.990249 
Minibatch iteration 5/19200: mean batch inertia: 1.977558, ewa inertia: 1.990116 
Minibatch iteration 6/19200: mean batch inertia: 2.156130, ewa inertia: 1.991853 
Minibatch iteration 7/19200: mean batch inertia: 1.886163, ewa inertia: 1.990747 
Minibatch iteration 8/19200: mean batch inertia: 2.106693, ewa inertia: 1.991960 
Minibatch iteration 9/19200: mean batch inertia: 1.896784, ewa inertia: 1.990964 
[MiniBatchKMeans] Reassigning 50 cluster centers.
Minibatch iteration 10/19200: mean batch inertia: 2.069250, ewa inertia: 1.991783 
Minibatch iteration 11/19200: mean batch inertia: 2.069137, ewa inertia: 1.992593 
Minibatch ite

...

Minibatch iteration 1216/19200: mean batch inertia: 1.722970, ewa inertia: 1.667046 
Minibatch iteration 1217/19200: mean batch inertia: 1.778233, ewa inertia: 1.668209 
Minibatch iteration 1218/19200: mean batch inertia: 1.828916, ewa inertia: 1.669891 
Minibatch iteration 1219/19200: mean batch inertia: 1.624877, ewa inertia: 1.669420 
Minibatch iteration 1220/19200: mean batch inertia: 1.524237, ewa inertia: 1.667901 
Minibatch iteration 1221/19200: mean batch inertia: 1.650642, ewa inertia: 1.667720 
Minibatch iteration 1222/19200: mean batch inertia: 1.620966, ewa inertia: 1.667231 
Minibatch iteration 1223/19200: mean batch inertia: 1.589223, ewa inertia: 1.666415 
Minibatch iteration 1224/19200: mean batch inertia: 1.758467, ewa inertia: 1.667378 
Minibatch iteration 1225/19200: mean batch inertia: 1.846395, ewa inertia: 1.669251 
Minibatch iteration 1226/19200: mean batch inertia: 1.434593, ewa inertia: 1.666796 
Minibatch iteration 1227/19200: mean batch inertia: 1.682138, ewa

Minibatch iteration 1323/19200: mean batch inertia: 1.926096, ewa inertia: 1.642051 
Minibatch iteration 1324/19200: mean batch inertia: 1.560157, ewa inertia: 1.641194 
Minibatch iteration 1325/19200: mean batch inertia: 1.508331, ewa inertia: 1.639804 
Minibatch iteration 1326/19200: mean batch inertia: 1.515477, ewa inertia: 1.638503 
Minibatch iteration 1327/19200: mean batch inertia: 1.495905, ewa inertia: 1.637011 
Minibatch iteration 1328/19200: mean batch inertia: 1.517921, ewa inertia: 1.635765 
Minibatch iteration 1329/19200: mean batch inertia: 1.616739, ewa inertia: 1.635566 
Minibatch iteration 1330/19200: mean batch inertia: 2.071460, ewa inertia: 1.640127 
Minibatch iteration 1331/19200: mean batch inertia: 1.634276, ewa inertia: 1.640066 
Minibatch iteration 1332/19200: mean batch inertia: 1.910381, ewa inertia: 1.642894 
[MiniBatchKMeans] Reassigning 50 cluster centers.
Minibatch iteration 1333/19200: mean batch inertia: 1.537167, ewa inertia: 1.641788 
Minibatch itera

Minibatch iteration 1418/19200: mean batch inertia: 1.676395, ewa inertia: 1.647504 
[MiniBatchKMeans] Reassigning 50 cluster centers.
Minibatch iteration 1419/19200: mean batch inertia: 1.532828, ewa inertia: 1.646304 
Minibatch iteration 1420/19200: mean batch inertia: 1.547658, ewa inertia: 1.645272 
Minibatch iteration 1421/19200: mean batch inertia: 1.684620, ewa inertia: 1.645683 
Minibatch iteration 1422/19200: mean batch inertia: 1.502199, ewa inertia: 1.644182 
Minibatch iteration 1423/19200: mean batch inertia: 1.719013, ewa inertia: 1.644965 
Minibatch iteration 1424/19200: mean batch inertia: 1.711598, ewa inertia: 1.645662 
Minibatch iteration 1425/19200: mean batch inertia: 1.824629, ewa inertia: 1.647535 
Minibatch iteration 1426/19200: mean batch inertia: 1.680498, ewa inertia: 1.647880 
Minibatch iteration 1427/19200: mean batch inertia: 1.523981, ewa inertia: 1.646583 
Minibatch iteration 1428/19200: mean batch inertia: 1.765092, ewa inertia: 1.647823 
Minibatch itera

Minibatch iteration 1512/19200: mean batch inertia: 1.804138, ewa inertia: 1.666333 
Minibatch iteration 1513/19200: mean batch inertia: 1.772648, ewa inertia: 1.667446 
Minibatch iteration 1514/19200: mean batch inertia: 1.401848, ewa inertia: 1.664667 
Minibatch iteration 1515/19200: mean batch inertia: 1.780598, ewa inertia: 1.665880 
Minibatch iteration 1516/19200: mean batch inertia: 1.521880, ewa inertia: 1.664373 
Minibatch iteration 1517/19200: mean batch inertia: 1.768808, ewa inertia: 1.665466 
Minibatch iteration 1518/19200: mean batch inertia: 1.732433, ewa inertia: 1.666166 
Minibatch iteration 1519/19200: mean batch inertia: 1.822607, ewa inertia: 1.667803 
Minibatch iteration 1520/19200: mean batch inertia: 1.455427, ewa inertia: 1.665581 
Minibatch iteration 1521/19200: mean batch inertia: 1.813568, ewa inertia: 1.667129 
Minibatch iteration 1522/19200: mean batch inertia: 1.648922, ewa inertia: 1.666939 
Minibatch iteration 1523/19200: mean batch inertia: 1.620867, ewa

Minibatch iteration 1616/19200: mean batch inertia: 1.512338, ewa inertia: 1.646639 
Minibatch iteration 1617/19200: mean batch inertia: 1.511483, ewa inertia: 1.645225 
Minibatch iteration 1618/19200: mean batch inertia: 1.531475, ewa inertia: 1.644035 
Minibatch iteration 1619/19200: mean batch inertia: 1.708943, ewa inertia: 1.644714 
Minibatch iteration 1620/19200: mean batch inertia: 1.690511, ewa inertia: 1.645193 
Minibatch iteration 1621/19200: mean batch inertia: 1.556093, ewa inertia: 1.644261 
Minibatch iteration 1622/19200: mean batch inertia: 1.752016, ewa inertia: 1.645388 
Minibatch iteration 1623/19200: mean batch inertia: 1.716163, ewa inertia: 1.646129 
Minibatch iteration 1624/19200: mean batch inertia: 1.562690, ewa inertia: 1.645256 
Minibatch iteration 1625/19200: mean batch inertia: 1.659091, ewa inertia: 1.645400 
Minibatch iteration 1626/19200: mean batch inertia: 1.729345, ewa inertia: 1.646279 
Minibatch iteration 1627/19200: mean batch inertia: 1.759122, ewa

In [217]:
ft.save('ft_200')

In [113]:
from sklearn.decomposition import NMF
import pandas as pd

In [119]:
vectorizer = TfidfVectorizer(max_features=25000, min_df=5, max_df=0.3, lowercase=False)
X = vectorizer.fit_transform([' '.join(t) for t in texts_norm])

In [127]:

model = NMF(n_components=100)

In [128]:
model.fit(X)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
  n_components=100, random_state=None, shuffle=False, solver='cd',
  tol=0.0001, verbose=0)

In [132]:

def get_nmf_topics(model, n_top_words):
    
    #id слов.
    feat_names = vectorizer.get_feature_names()
    
    word_dict = {};
    for i in range(100):
        
        #топ n слов для темы.
        words_ids = model.components_[i].argsort()[:-n_top_words - 1:-1]
        words = [feat_names[key] for key in words_ids]
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = words;
    
    return pd.DataFrame(word_dict), word_dict

In [133]:
_, d = get_nmf_topics(model, 10)

In [134]:
d

{'Topic # 01': ['so',
  'okay',
  'let',
  'here',
  'about',
  'just',
  'these',
  'right',
  'what',
  'are'],
 'Topic # 02': ['курс',
  'семестр',
  'учиться',
  'студент',
  'плюс',
  'задание',
  'яндекс',
  'проходить',
  'shot',
  'сдавать'],
 'Topic # 03': ['вопрос',
  'спасибо',
  'доклад',
  'пожалуйста',
  'ответить',
  'задать',
  'ответ',
  'задавать',
  'ваш',
  'здравствовать'],
 'Topic # 04': ['you',
  'can',
  'have',
  'if',
  'know',
  'your',
  'do',
  'get',
  'see',
  're'],
 'Topic # 05': ['модель',
  'строить',
  'линейный',
  'построить',
  'обучить',
  'параметр',
  'ансамбль',
  'предсказывать',
  'обучать',
  'различный'],
 'Topic # 06': ['we',
  'can',
  'have',
  'our',
  'for',
  'this',
  'do',
  'data',
  'use',
  're'],
 'Topic # 07': ['the',
  'is',
  'same',
  'at',
  'on',
  'from',
  'of',
  'layer',
  'to',
  'first'],
 'Topic # 08': ['дать',
  'анализ',
  'база',
  'случай',
  'обработка',
  'данный',
  'набор',
  'собирать',
  'вид',
  'реальны

In [189]:
words = Counter()
for text in texts_norm:
    words.update(text.split())

In [190]:
words.most_common()

[('и', 81049),
 ('быть', 72003),
 ('в', 54630),
 ('что', 50744),
 ('это', 50526),
 ('то', 50141),
 ('вот', 49765),
 ('мы', 49357),
 ('на', 44570),
 ('не', 40206),
 ('the', 37148),
 ('этот', 30714),
 ('как', 29335),
 ('который', 27708),
 ('я', 26363),
 ('там', 22769),
 ('но', 21829),
 ('с', 21052),
 ('вы', 21016),
 ('они', 20287),
 ('у', 20080),
 ('такой', 20045),
 ('and', 19942),
 ('он', 19266),
 ('of', 19185),
 ('ну', 17645),
 ('to', 17501),
 ('по', 16707),
 ('is', 16262),
 ('you', 15236),
 ('a', 15144),
 ('так', 14872),
 ('so', 14662),
 ('that', 14494),
 ('для', 14362),
 ('мочь', 14316),
 ('если', 14284),
 ('а', 13991),
 ('this', 12795),
 ('все', 12712),
 ('очень', 12687),
 ('we', 12462),
 ('весь', 12445),
 ('it', 11578),
 ('из', 11206),
 ('бы', 11193),
 ('просто', 11081),
 ('in', 10914),
 ('можно', 10586),
 ('она', 10169),
 ('один', 9613),
 ('самый', 9524),
 ('I', 9444),
 ('еще', 9329),
 ('тот', 8643),
 ('s', 8636),
 ('или', 8386),
 ('же', 8320),
 ('к', 7858),
 ('да', 7855),
 ('здес